# den Haan, Ramey and Watson (2000)

## The Model

### Firms' value function

$$
V(a) = \max \{ profit + \beta E [V(a')], 0 \}
$$

### Value of employed

$$
V_e = \text{wage} + \beta E \left[  \text{proba of remaining employed} \cdot V'_e + \text{proba of becoming unemployed} \cdot V' _u  \right]
$$

### Value of unemployed

$$
V_u = \text{unemployed benefits} + \beta E[ \text{proba of finding employment} \cdot V'_e +  \text{proba of remaining unemployed} \cdot V'_u ]
$$

## Pseudo code for numerical solution using value function iteration

In [2]:
##### Source: generated using Google's Gemini 2.0 Flash (8/3/2025)

# 1. Initialization

# Define state space (e.g., productivity levels, employment status)
state_space = [...]

# Define parameters (discount factor, etc.)
beta = ...
probability_employed = ...  # Function or matrix
probability_unemployed = ... # Function or matrix
wage_function = ...
profit_function = ...
unemployment_benefits = ...

# Initialize value functions (e.g., with zeros)
value_firm = {state: 0 for state in state_space} # Dictionary: state -> value
value_employed = {state: 0 for state in state_space}
value_unemployed = {state: 0 for state in state_space}

# Set convergence tolerance and maximum iterations
tolerance = 1e-6
max_iterations = 1000
iteration = 0
distance = float('inf')

# 2. Value Function Iteration

while distance > tolerance and iteration < max_iterations:
    iteration += 1
    new_value_firm = {}
    new_value_employed = {}
    new_value_unemployed = {}
    distance = 0

    for state in state_space:
        # Firm's Bellman equation
        profit = profit_function(state)
        expected_future_value_firm = 0 #Calculate expected future value.
        #Code would need to be added here to calculate the expected future value.
        new_value_firm[state] = max(profit + beta * expected_future_value_firm, 0) # 0 represents job destruction.
        distance = max(distance, abs(new_value_firm[state] - value_firm[state]))

        # Worker's Bellman equations
        if worker_is_employed(state): #function that returns a boolean.
            wage = wage_function(state)
            expected_future_value_employed = 0 #Calculate expected future value.
            expected_future_value_unemployed = 0 #Calculate expected future value.
            #Code would need to be added here to calculate the expected future values.
            new_value_employed[state] = wage + beta * (probability_employed(state) * expected_future_value_employed +
                                                      (1 - probability_employed(state)) * expected_future_value_unemployed)
            distance = max(distance, abs(new_value_employed[state] - value_employed[state]))
        else: # worker is unemployed.
            benefits = unemployment_benefits(state)
            expected_future_value_employed = 0 #Calculate expected future value.
            expected_future_value_unemployed = 0 #Calculate expected future value.
            #Code would need to be added here to calculate the expected future values.
            new_value_unemployed[state] = benefits + beta * (probability_unemployed(state) * expected_future_value_employed +
                                                        (1 - probability_unemployed(state)) * expected_future_value_unemployed)
            distance = max(distance, abs(new_value_unemployed[state] - value_unemployed[state]))

    # Update value functions
    value_firm = new_value_firm
    value_employed = new_value_employed
    value_unemployed = new_value_unemployed

# 3. Policy Function Extraction

# Based on the converged value functions, extract policy functions (e.g., job destruction decisions)
def job_destruction_policy(state):
    profit = profit_function(state)
    expected_future_value_firm = 0 #Calculate expected future value.
    #Code would need to be added here to calculate the expected future value.
    if profit + beta * expected_future_value_firm < 0:
        return True  # Destroy job
    else:
        return False # Continue job

TypeError: 'ellipsis' object is not callable

## Original Fortran code

### Common Variables
- **Common Block `/c1/`:** Contains arrays and parameters related to state transitions, economic variables (like consumption, wages), and model settings.
- **Common Block `/i1/`:** Includes indices and flags for iteration limits and creation status.

### Functions

#### `equil(trlog)`
- **Purpose:** Computes the equilibrium condition based on a given shock level (`trlog`).
- **Key Steps:**
  - Calculate the sum of exponential shocks weighted by their probability density.
  - Compute rental rate using capital intensity and labor productivity.
  - Determine truncation level for idiosyncratic shocks, ensuring it doesn't fall below a threshold.
  - Return the difference between calculated and input shock levels.

#### `equilcap(rr)`
- **Purpose:** Calculates equilibrium interest rates based on given parameters (`rr`).
- **Key Steps:**
  - Check if certain economic conditions are met; adjust truncation level accordingly.
  - Compute capital allocation using a derived formula involving rental rate, productivity, and interest rate.
  - Sum up allocated capital across possible shock levels.
  - Return the difference between total capital and summed allocations.

### Key Considerations

- **Numerical Stability:** Ensure that calculations do not result in extremely low or high probability events. This is handled by checking bounds for `trunc`.
- **Parameter Sensitivity:** Parameters like `alpha`, `the`, and `bvalue` significantly influence the model's behavior.
- **Optimization:** The use of numerical methods (e.g., Simpson's rule) suggests an optimization problem, likely solved iteratively.


### drw3net.f (basic program used to derive the policy rules)

In [3]:
*
*       drw3net.f = drw3.f without the specification of the subdirectories
*
*       drw3.f, a rewritten version of vac5.f with additional
*       subroutines to calculate the steady state, to check for
*       accuracy, and to calculate summary statistics
*
*       vac5.f, like vac4.f but with exogenous separation before
*       and after production takes place.
*
*       vac4.f, like vac3.f but with new matching function
*
*       vac3.f, like vac1.f but with variable elasticity in the
*       matching function
*
*       vac2.f, program with exogenous separation after production
*       has taken place
*
*       vac1.f, with endogenous vacancies and exogenous seperation
*
*       mp1.f, like d1.f but positive value of being in the matching pool
*
*       d1.f, like depr1.f but only parameterizing the difference
*       of being matched and being in the pool
*
*       depr1.f, complete basis
*
*       depr2.f, capital choice is made before idiosyncratic shock
*       is observed
*


	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap                                       
	
	dimension root(3000),roothx(1000),roothw(1000),rootlx(1000),
     $  rootlw(1000)

	external fdens,equil,equilcap


c
c       npar is number of parameters for the policy functions
c

	open(33,file='drwcoef.dat',status='unknown')
	open(34,file='drwcoef.new',status='unknown')
	open(41,file='drw.dat',status='unknown')
	open(36,file='cheb0.dat',status='unknown')
	open(39,file='herm.dat',status='unknown')
	open(40,file='herm.pro',status='unknown')
	open(55,file='for055.dat',status='unknown')
	open(99,file='for099.dat',status='unknown')
c
c       read chebyshev and hermite nodes
c

	read(36,*) (root(j),j=1,870)
	read(39,*) (roothx(j),j=1,780)
	read(40,*) (roothw(j),j=1,780)

	read(33,*) ((aold(j,k),j=1,npar),k=1,4)

	read(41,*) i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,
     $  alpha,bta,depr,gam,ro,sig,xvalue,pmatch,eps,
     $  bl1,bh1,bl3,bh3,fstan,acc,xacc,itotal,i2bound,bvalue,share,
     $  prob,eta,auto2,abba,auto1,istst,it1,it2,it3,imom,icreate,
     $  impulse,icap
	
	pi       = 3.141592654
	
c
c
c       construct Simpson weights (note that the number of Simpson
c       nodes has to be odd)
c
c

	hsw(1)        = 1./3.
	hsw(nsimpson) = 1./3.
	itel = -1
	do 905 i = 2,nsimpson-1
	if(itel.lt.0) then
	hsw(i) = 4./3.
	else
	hsw(i) = 2./3.
	endif
	itel = -1*itel
905     continue

	write(*,'(10f7.3)') (( aold(i,kk),i=1,npar),kk=1,4)

	if(nhermite.gt.30) then
	write(*,*) 'not enough nodes in herm.dat and herm.pro' 
	stop
	endif


	do 910 i = 1,nhermite
	istart = (nhermite-5)*30
	hhx(i) = roothx(istart+i)
	hhw(i) = roothw(istart+i)
910     continue


c
c       numerically calculate the integral that will be useful 
c       for calculating the interest rate
c

	asum = 0.

	yy = (8.*fstan)/dble(nsimpson-1.)

	do  1911 i = 1,nsimpson
	qx =  dble(i-1)*yy  - 4.*fstan
	qw =  hsw(i)*yy*fdens(qx)
	asum = asum + exp(qx)*qw
1911    continue
	
c
c       calculate the steady state
c

	capstst  =  (1.-bta*(1.-depr))/(bta*alpha*asum)
	capstst  =  capstst**(1./(alpha-1.))
	write(*,*) 'capstst (incorporates asum)', capstst        

c
c       Construct Gridpoints 
c
c       stbar(.,.) contains the boundaries
c

	stbar(1,1) = bl1
	stbar(1,2) = bh1
	stbar(2,1) = -i2bound*(sig**2./(1.-ro**2.))**0.5
	stbar(2,2) = -stbar(2,1)
	stbar(3,1) = bl3
	stbar(3,2) = bh3

c
c       below we first find the appropriate roots, and then
c       find the corresponding unscaled numbers
c

	do 10 is1 = 1,is1max
	istart = (is1max-2)*30
	st(is1,1) = root(istart+is1)
10      continue
	do 20 is1 = 1,is1max
	bb = st(is1,1)
	stu(is1,1) = scinv(bb,1)
20      continue

	do 11 is2 = 1,is2max
	istart = (is2max-2)*30
	st(is2,2) = root(istart+is2)
11      continue
	do 21 is2 = 1,is2max
	bb = st(is2,2)
	stu(is2,2) = scinv(bb,2)
21      continue


	do 12 is3 = 1,is3max
	istart = (is3max-2)*30
	st(is3,3) = root(istart+is3)
12      continue
	do 22 is3 = 1,is3max
	bb = st(is3,3)
	stu(is3,3) = scinv(bb,3)
22      continue
 
c
c       Construct the chebyshev polynomials at the grid points
c       (Since these have to be calculated many times, it is
c       worthwhile to save them
c

	do 40 is3 = 1,is3max
	do 40 is2 = 1,is2max
	do 40 is1 = 1,is1max
	jt  = ispoint(is1,is2,is3)
	ss1 = st(is1,1)
	ss2 = st(is2,2)
	ss3 = st(is3,3)
	do 41 j1 = 1,i1max+1
	do 41 j2 = 1,i2max+1
	do 41 j3 = 1,i3max+1
	i1 = j1-1
	i2 = j2-1
	i3 = j3-1
	j = ipoint(i1,i2,i3)
	cheb(jt,j) = hh(ss1,ss2,ss3,i1,i2,i3)
41      continue
40      continue

	write(*,*) ipoint(i1max,i2max,i3max),npar
	

	jmax = is1max*is2max*is3max

	DO 110 J = 1,NPAR
	XX(J,J) = 0.0
110     CONTINUE
	
	DO 130 J = 1,NPAR
	DO 130 I = 1,JMAX
	XX(J,J) = XX(J,J) + CHEB(I,J)*CHEB(I,J)
130     CONTINUE

c
c
c       note that the orthogonality property of the chebyshev polynomial
c       is used to calculate the inverse of the x'x matrix
c
c

	DO 135 J = 1,NPAR        
	XX(J,J) = 1./XX(J,J)
135     CONTINUE

c
c
c       Iterating on Consumer Problem
c
c

	do 300 iter = 1,itotal
 
	call sbta1()
	

	accur = 0.
	do 310 kk  = 1,4
	do 310 i = 1,npar
	if(abs(aold(i,kk)).gt.0.001) then
	accur = accur + abs( (aold(i,kk)-anew(i,kk))/aold(i,kk)  )
	else
	accur = accur + abs( (aold(i,kk)-anew(i,kk)) )
	endif
310     continue

	write(*,*) accur,iter
	
	if(accur.lt.acc.or.iter.eq.itotal) goto 305
	
	do 311 kk = 1,4
	do 311  i = 1,npar
	aold(i,kk) = eps*anew(i,kk) + (1.-eps)*aold(i,kk)
311     continue

	
300     continue
305     continue

	do 600 kk = 1,4
	write(34,*) (aold(j,kk),j=1,npar)
600     continue

	if(istst.eq.1) call sbta2()
	  if(impulse.eq.1) call sbta3()
	if(it1.eq.1)   call sacc1()
	if(it2.eq.1)   call sacc2()
	if(it3.eq.1)   call sacc3()
	if(imom .eq.1) call sbta4()
	
	stop
	end

***************************************************************************
*
*       Sbta1(); Subroutine to calculate at each grid point numerically
*       the four conditional expectations and to project the calculated
*       values on the Chebyshev polynomial terms
*
***************************************************************************

	subroutine sbta1()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap
	
	dimension y(20000,4),xy(4000)

	external fdens,equil,equilcap
	
	do 10 is1 = 1,is1max
	do 10 is2 = 1,is2max
	do 10 is3 = 1,is3max

	jt = ispoint(is1,is2,is3)

	s1old = st(is1,1)
	s2old = st(is2,2)
	s3old = st(is3,3)

	pemployed = stu(is3,3) 
	cap  =   stu(is1,1)
	the  =   exp(stu(is2,2))

	gg    =    polgrid(is1,is2,is3,2)
	ww    =    polgrid(is1,is2,is3,3)
	wf    =    polgrid(is1,is2,is3,4)
	exp1  =    polgrid(is1,is2,is3,1)

	call poltrunc()
	call polprod()

	expold    = exp1
	theold    = the
	cap       = capnew
	pemployed = pempnew

	call polexp()


	y(jt,1) = log(expcons)
	y(jt,2) = log(expmatch)
	y(jt,3) = log(expwork)
	y(jt,4) = log(expfirm)

10      continue
		
	JMAX = IS1MAX*IS2MAX*IS3MAX
	
	do 600 kk = 1,4                                   

	DO 110 J = 1,NPAR   
	XY(J) = 0.0
110     CONTINUE

	DO 140 J = 1,NPAR
	DO 140 I = 1,JMAX
	XY(J) = XY(J) + CHEB(I,J)*Y(I,kk)
140     CONTINUE
	
	DO 145 J = 1,NPAR
	ANEW(J,KK) =  XX(J,J)*XY(J)
145     CONTINUE

600     continue

	end

***************************************************************************
*
*       Sbta2(); Subroutine to calculate the steady state values
*
***************************************************************************

	subroutine sbta2()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap
	
	external fdens,equil,equilcap

	pemployed   = stu(2,3)
	cap         = stu(2,1)
	theold      = exp(stu(2,2))

	idum = -200597
		
	do 10 itime = 1,10000

	shock = 0.
c       write(*,*) cap,pemployed

	the  = ro*log(theold)+shock*sig
	s2old = sca(the,2)
	the  = exp(the)
	s1old = sca(cap,1)
	s3old = sca(pemployed,3)

	gg    =  pol(s1old,s2old,s3old,2)
	ww    =  pol(s1old,s2old,s3old,3)
	wf    =  pol(s1old,s2old,s3old,4)
	exp1  =  pol(s1old,s2old,s3old,1)

	call poltrunc()
	call polprod()

	accz = abs(pempnew-pemployed)+abs(truncpr-old2)
     $      + abs(pmatch-old3)+abs(vacpr-old4)
	  
	
	
	old2 = truncpr
	old3 = pmatch
	old4 = vacpr

	if(itime.eq.1000.or.accz.le.0.0001) then
	write(*,701) unemp/pemployed
	write(*,702) vac/pemployed
	write(*,703) truncpr        
	write(*,704) pmatch
	write(*,705) vacpr
	write(*,706) bvalue
	write(*,707) xvalue
	write(*,708) prob
	write(*,709) eta
	write(*,710) fstan
	
	write(55,701) unemp/pemployed
	write(55,702) vac/pemployed
	write(55,703) truncpr        
	write(55,704) pmatch
	write(55,705) vacpr
	write(55,706) bvalue
	write(55,707) xvalue
	write(55,708) prob
	write(55,709) eta
	write(55,710) fstan
	
	goto 11
	endif


*
*       calculate next period's fraction of people employed
*

	pemployed   = pempnew
	cap         = capnew
	theold      = the

10      continue
11      continue                                        
		
701     format('un/pemployed:  ',f14.6)
702     format('vac/pemployed: ',f14.6)
703     format('truncpr:       ',f14.6)
704     format('pmatch:        ',f14.6)
705     format('pvac:          ',f14.6)
706     format('bvalue:        ',f14.6)
707     format('xvalue:        ',f14.6)    
708     format('prob:          ',f14.6)
709     format('eta:           ',f14.6)
710     format('fstan:         ',f14.6)

	end

***************************************************************************
*
*       Sbta3(); Subroutine to calculate impulse response function
*
***************************************************************************

	subroutine sbta3()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap
	
	external fdens,equil,equilcap


	open(31,file='impu.dat',
     $status='unknown')

	pemployed   = stu(is3max,3)
	cap         = stu(is1max,1)
	theold      = exp(stu(is2max,2))

	idum = -200597
		
	do 10 itime = 1,1000
	
	shock = 0.
	  
	if(itime.eq.800) shock = -3.

	the  = ro*log(theold)+shock*sig
	s2old = sca(the,2)
	the  = exp(the)
	s1old = sca(cap,1)
	s3old = sca(pemployed,3)

	gg    =  pol(s1old,s2old,s3old,2)
	ww    =  pol(s1old,s2old,s3old,3)
	wf    =  pol(s1old,s2old,s3old,4)
	exp1  =  pol(s1old,s2old,s3old,1)

	call poltrunc()
	call polprod()

	  
	rotot  =  1.-auto1*auto2*truncpr
	roex   =  truncpr*(1.-auto1*auto2)
	repost =  roex
	

		
	yx2 = pempnew
	    yx3 = (rotot - repost*vacpr)
     $           * 2.*pemployed/(pemployed+pempnew)
	    yx4 = vacpr*((vac/pemployed)-repost)
     $           * 2.*pemployed/(pemployed+pempnew)
	
			
	rentadj = dble(icap)*truncpr*auto1 + (1.-dble(icap))*1.
	efrental = rentadj*rental
      
       if(itime.ge.781) then
       write(31,'(i6,17f12.7)') itime,the,prod,yx2,yx3,yx4,
     $ pmatch,vacpr,vac,cap,
     $ efrental,rental,truncpr,cons1,bta*consold/cons1,
     $ gg,ww,wf
       endif


*       calculate next period's fraction of people employed
*

	pemployed   = pempnew
	cap         = capnew
	theold      = the
	consold     = cons1

10      continue
11      continue                                        
	

	  end


***************************************************************************
*
*       SACC1(); Subroutine to check for accuracy for a range of
*       different values for the capital stock
*
***************************************************************************

	subroutine sacc1()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap
	

	external fdens,equil,equilcap

	open(91,file
     $  ='for091.dat',status='unknown')
	open(92,file
     $  ='for092.dat',status='unknown')
	open(93,file
     $  ='for093.dat',status='unknown')
	open(94,file
     $  ='for094.dat',status='unknown')
	open(95,file
     $  ='for095.dat',status='unknown')
	open(96,file
     $  ='for096.dat',status='unknown')
	open(97,file
     $  ='for097.dat',status='unknown')
	open(98,file
     $  ='for098.dat',status='unknown')

	do 10 is2 = 1,is2max
	do 10 is3 = 1,is3max
	write(*,*) is2,is3
	do 10 is1 = 1,51

	cap  = stbar(1,1)+dble(is1-1)*(stbar(1,2)-stbar(1,1))/51.
	s1old = sca(cap,1)

	s2old = st(is2,2)
	s3old = st(is3,3)

	pemployed = stu(is3,3) 
	the  =   exp(stu(is2,2))


	exp1  =    pol(s1old,s2old,s3old,1)
	gg    =    pol(s1old,s2old,s3old,2)
	ww    =    pol(s1old,s2old,s3old,3)
	wf    =    pol(s1old,s2old,s3old,4)

	pol1  =    exp1
	pol2  =    gg
	pol3  =    ww
	pol4  =    wf

	call poltrunc()
	call polprod()

	expold    = exp1
	theold    = the
	cap       = capnew
	pemployed = pempnew

	call polexp()

	write(91,'(f14.6)')  pol1
	write(92,'(f14.6)')  pol2
	write(93,'(f14.6)')  pol3
	write(94,'(f14.6)')  pol4

	write(95,'(f14.6)')  expcons
	write(96,'(f14.6)')  expmatch
	write(97,'(f14.6)')  expwork
	write(98,'(f14.6)')  expfirm



10      continue

	close(unit=91)
	close(unit=92)
	close(unit=93)
	close(unit=94)
	close(unit=95)
	close(unit=96)
	close(unit=97)
	close(unit=98)

	end

***************************************************************************
*
*       SACC2(); Subroutine to check for accuracy for a range of
*       different values for the productivity shock
*
***************************************************************************

	subroutine sacc2()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap
	

	external fdens,equil,equilcap

	open(81,file
     $  ='for081.dat',status='unknown')
	open(82,file
     $  ='for082.dat',status='unknown')
	open(83,file
     $  ='for083.dat',status='unknown')
	open(84,file
     $  ='for084.dat',status='unknown')
	open(85,file
     $  ='for085.dat',status='unknown')
	open(86,file
     $  ='for086.dat',status='unknown')
	open(87,file
     $  ='for087.dat',status='unknown')
	open(88,file
     $  ='for088.dat',status='unknown')

	do 10 is1 = 1,is1max
	do 10 is3 = 1,is3max

	write(*,*) is1,is3
	do 10 is2 = 1,51


	the   = stbar(2,1)+dble(is2-1)*(stbar(2,2)-stbar(2,1))/51.
	s2old = sca(the,2)

	s1old = st(is1,1)
	s3old = st(is3,3)

	pemployed = stu(is3,3)
	cap       = stu(is1,1)

	the  =   exp(the)


	exp1  =    pol(s1old,s2old,s3old,1)
	gg    =    pol(s1old,s2old,s3old,2)
	ww    =    pol(s1old,s2old,s3old,3)
	wf    =    pol(s1old,s2old,s3old,4)


	pol1  =    exp1
	pol2  =    gg
	pol3  =    ww
	pol4  =    wf

	call poltrunc()
	call polprod()

	expold    = exp1
	theold    = the
	cap       = capnew
	pemployed = pempnew

	call polexp()

	write(81,'(f14.6)')  pol1
	write(82,'(f14.6)')  pol2
	write(83,'(f14.6)')  pol3
	write(84,'(f14.6)')  pol4

	write(85,'(f14.6)')  expcons
	write(86,'(f14.6)')  expmatch
	write(87,'(f14.6)')  expwork
	write(88,'(f14.6)')  expfirm



10      continue
		
	close(unit=81)
	close(unit=82)
	close(unit=83)
	close(unit=84)
	close(unit=85)
	close(unit=86)
	close(unit=87)
	close(unit=88)

	end

***************************************************************************
*
*       SACC3(); Subroutine to check for accuracy for a range of
*       different values for the fraction of employed matches
*
***************************************************************************

	subroutine sacc3()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap
	

	external fdens,equil,equilcap


	open(71,file
     $  ='for071.dat',status='unknown')
	open(72,file
     $  ='for072.dat',status='unknown')
	open(73,file
     $  ='for073.dat',status='unknown')
	open(74,file
     $  ='for074.dat',status='unknown')
	open(75,file
     $  ='for075.dat',status='unknown')
	open(76,file
     $  ='for076.dat',status='unknown')
	open(77,file
     $  ='for077.dat',status='unknown')
	open(78,file
     $  ='for078.dat',status='unknown')
	
	do 10 is2 = 1,is2max
	do 10 is1 = 1,is1max

	write(*,*) is2,is1

	do 10 is3 = 1,51

	pemployed = stbar(3,1)+dble(is3-1)*(stbar(3,2)-stbar(3,1))/51.
	s3old = sca(pemployed,3)

	s2old = st(is2,2)
	s1old = st(is1,1)

	cap  =   stu(is1,1) 
	the  =   exp(stu(is2,2))

	exp1  =    pol(s1old,s2old,s3old,1)
	gg    =    pol(s1old,s2old,s3old,2)
	ww    =    pol(s1old,s2old,s3old,3)
	wf    =    pol(s1old,s2old,s3old,4)

	pol1  =    exp1
	pol2  =    gg
	pol3  =    ww
	pol4  =    wf

	call poltrunc()
	call polprod()

	expold    = exp1
	theold    = the
	cap       = capnew
	pemployed = pempnew

	call polexp()

	write(71,'(f14.6)')  pol1
	write(72,'(f14.6)')  pol2
	write(73,'(f14.6)')  pol3
	write(74,'(f14.6)')  pol4

	write(75,'(f14.6)')  expcons
	write(76,'(f14.6)')  expmatch
	write(77,'(f14.6)')  expwork
	write(78,'(f14.6)')  expfirm



10      continue

	close(unit=71)
	close(unit=72)
	close(unit=73)
	close(unit=74)
	close(unit=75)
	close(unit=76)
	close(unit=77)
	close(unit=78)
		

	end

***************************************************************************
*
*       Sbta4(); Subroutine to calculate the summary statistics
*
***************************************************************************

	subroutine sbta4()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap
	common/stat/ y(500,10),cor1(2,9),cor2(2,9),ave(10),stan(10),
     $  cor3(2,9),autocor(10,10),nobs,idump
	dimension cor1ave(2,9),cor2ave(2,9),cor3ave(1,9),ratave(10),
     $  autoave(10,10),hpy(150),hpt(150),hpd(150),hpv(150,3)
	dimension cor1se(2,9),cor2se(2,9),cor3se(1,9),ratse(10),
     $  autose(10,10)

	external fdens,equil,equilcap

	idum = -200597

	montetot = 100
	s     = 1600
	nobs  =  67
	idump = 200
	nw    = 3
	iopt  = 0


	do 211 jj = 1,8
	ratave(jj) = 0.
	ratse(jj)  = 0.
211     continue

	do 212 jj = 1,2
	do 212 kk = 1,9
	cor1ave(jj,kk) = 0.
	cor2ave(jj,kk) = 0.
	cor3ave(1,kk)  = 0.
	cor1se(jj,kk)  = 0.
	cor2se(jj,kk)  = 0.
	cor3se(jj,kk)  = 0.
212     continue
							     
	do 213 jj = 1,8
	do 213 kk = 1,4
	autoave(jj,kk) = 0.
	autose(jj,kk)  = 0.
213     continue

	do 115 monte = 1,montetot

	write(*,*) monte

	pemployed = stu(2,3)
	cap       = stu(2,1)
	theold    = exp(stu(2,2))

	do 10 itime  = 1,idump+nobs

	shock = gasdev(idum)

	the   = ro*log(theold)+shock*sig
	s2old = sca(the,2)
	the   = exp(the)
	s1old = sca(cap,1)
	s3old = sca(pemployed,3)

	gg    =  pol(s1old,s2old,s3old,2)
	ww    =  pol(s1old,s2old,s3old,3)
	wf    =  pol(s1old,s2old,s3old,4)
	exp1  =  pol(s1old,s2old,s3old,1)

	call poltrunc()
	call polprod()

	
	rotot  =  1.-auto1*auto2*truncpr
	roex   =  truncpr*(1.-auto1*auto2)
	repost =  roex

	y(itime,1) = prod
	y(itime,2) = pempnew
	y(itime,3) = (rotot - repost*vacpr)
     $           * 2.*pemployed/(pemployed+pempnew)
	y(itime,4) = vacpr*((vac/pemployed)-repost)
     $           * 2.*pemployed/(pemployed+pempnew)
	y(itime,5) = cons1
	y(itime,6) = capnew-(1.-depr)*cap      
	y(itime,7) = prod/cap**alpha
	y(itime,8) = the
	

	if(itime.gt.idump) then
	do 444 ikk = 1,8
	if(y(itime,ikk).le.0.) then
	write(*,*) itime,ikk
	write(99,*) itime,ikk
	y(itime,ikk) = 0.0001
	endif
	y(itime,ikk) = log(y(itime,ikk))
444     continue
	endif

	pemployed = pempnew
	cap       = capnew
	theold    = the      
	

10      continue



	do 900 j = 1,8
	do 910 i = 1,nobs
	hpy(i) = y(i+idump,j)
910     continue
	call hpfilt(hpy,hpt,hpd,hpv,nobs,nw,s,iopt)
	do 920 i = 1,nobs
	y(i+idump,j) = hpd(i)
920     continue
900     continue

	call avesub()
	call stansub()
	call corsub()
	call autosub()

	ratave(1) = ratave(1) + stan(1)
	ratave(2) = ratave(2) + stan(2)/stan(1)
	ratave(3) = ratave(3) + stan(3)/stan(2)
	ratave(4) = ratave(4) + stan(4)/stan(2)
	ratave(8) = ratave(8) + stan(1)/stan(8)
	ratse(1)  = ratse(1)  + stan(1)**2.
	ratse(2)  = ratse(2)  + (stan(2)/stan(1))**2.
	ratse(3)  = ratse(3)  + (stan(3)/stan(2))**2.
	ratse(4)  = ratse(4)  + (stan(4)/stan(2))**2.
	ratse(8)  = ratse(8)  + (stan(1)/stan(8))**2.
	
	do 11 jj = 5,7
	ratave(jj) = ratave(jj) +  stan(jj)/stan(1)
	ratse(jj)  = ratse(jj)  + (stan(jj)/stan(1))**2.
11      continue

	do 12 kk = 1,9
	cor3ave(1,kk)  = cor3ave(1,kk)  + cor3(1,kk)
	cor3se(1,kk)   = cor3se(1,kk)   + cor3(1,kk)**2.
	do 12 jj = 1,2
	cor1ave(jj,kk) = cor1ave(jj,kk) + cor1(jj,kk)
	cor2ave(jj,kk) = cor2ave(jj,kk) + cor2(jj,kk)
	cor1se(jj,kk)  = cor1se(jj,kk)  + cor1(jj,kk)**2.
	cor2se(jj,kk)  = cor2se(jj,kk)  + cor2(jj,kk)**2.
12      continue

	do 13 jj = 1,8
	do 13 kk = 1,4
	autoave(jj,kk) = autoave(jj,kk) + autocor(jj,kk)
	autose(jj,kk)  = autose(jj,kk)  + autocor(jj,kk)**2.
13      continue
115     continue

	do 111 jj = 1,8
	ratave(jj) = ratave(jj)/dble(montetot)
	ratse(jj)  = ratse(jj)/dble(montetot)
	ratse(jj)  = (ratse(jj)-ratave(jj)**2.)**0.5
111     continue

	do 112 kk = 1,9
	cor3ave(1,kk)  = cor3ave(1,kk)/dble(montetot)
	cor3se(1,kk)   = cor3se(1,kk)/dble(montetot)
	cor3se(1,kk)   = (cor3se(1,kk)-cor3ave(1,kk)**2.)**0.5
	do 112 jj = 1,2
	cor1ave(jj,kk) = cor1ave(jj,kk)/dble(montetot)
	cor2ave(jj,kk) = cor2ave(jj,kk)/dble(montetot)
	cor1se(jj,kk)  = cor1se(jj,kk)/dble(montetot)
	cor2se(jj,kk)  = cor2se(jj,kk)/dble(montetot)
	cor1se(jj,kk)  = (cor1se(jj,kk)-cor1ave(jj,kk)**2.)**0.5
	cor2se(jj,kk)  = (cor2se(jj,kk)-cor2ave(jj,kk)**2.)**0.5     
112   continue

	do 113 jj = 1,8
	do 113 kk = 1,4
	autoave(jj,kk) = autoave(jj,kk)/dble(montetot)
	autose(jj,kk)  = autose(jj,kk)/dble(montetot)
	autose(jj,kk)  = (autose(jj,kk)-autoave(jj,kk)**2.)**0.5     
113   continue

	write(*,701) ratave(1)
	write(*,702) ratave(2)
	write(*,703) ratave(3)
	write(*,704) ratave(4)
	write(*,705) ratave(5)
	write(*,706) ratave(6)
	write(*,707) ratave(7)
	write(*,708) ratave(8)

	write(*,801) (autoave(1,kk),kk=1,4)
	write(*,802) (autoave(2,kk),kk=1,4)
	write(*,803) (autoave(3,kk),kk=1,4)
	write(*,804) (autoave(4,kk),kk=1,4)
	write(*,805) (autoave(5,kk),kk=1,4)
	write(*,806) (autoave(6,kk),kk=1,4)
	write(*,807) (autoave(7,kk),kk=1,4)
	write(*,808) (autoave(8,kk),kk=1,4)

	write(*,721) (cor1ave(1,kk),kk=1,9)
	write(*,722) (cor1ave(2,kk),kk=1,9)
	write(*,723) (cor2ave(1,kk),kk=1,9)
	write(*,724) (cor2ave(2,kk),kk=1,9)
	write(*,725) (cor3ave(1,kk),kk=1,9)

	write(55,701) ratave(1)
	write(55,702) ratave(2)
	write(55,703) ratave(3)
	write(55,704) ratave(4)
	write(55,705) ratave(5)
	write(55,706) ratave(6)
	write(55,707) ratave(7)
	write(55,708) ratave(8)

	write(55,801) (autoave(1,kk),kk=1,4)
	write(55,802) (autoave(2,kk),kk=1,4)
	write(55,803) (autoave(3,kk),kk=1,4)
	write(55,804) (autoave(4,kk),kk=1,4)
	write(55,805) (autoave(5,kk),kk=1,4)
	write(55,806) (autoave(6,kk),kk=1,4)
	write(55,807) (autoave(7,kk),kk=1,4)
	write(55,808) (autoave(8,kk),kk=1,4)

	write(55,721) (cor1ave(1,kk),kk=1,9)
	write(55,722) (cor1ave(2,kk),kk=1,9)
	write(55,723) (cor2ave(1,kk),kk=1,9)
	write(55,724) (cor2ave(2,kk),kk=1,9)
	write(55,725) (cor3ave(1,kk),kk=1,9)
	
	write(*,701) ratse(1)
	write(*,702) ratse(2)
	write(*,703) ratse(3)
	write(*,704) ratse(4)
	write(*,705) ratse(5)
	write(*,706) ratse(6)
	write(*,707) ratse(7)
	write(*,708) ratse(8)

	write(*,801) (autose(1,kk),kk=1,4)
	write(*,802) (autose(2,kk),kk=1,4)
	write(*,803) (autose(3,kk),kk=1,4)
	write(*,804) (autose(4,kk),kk=1,4)
	write(*,805) (autose(5,kk),kk=1,4)
	write(*,806) (autose(6,kk),kk=1,4)
	write(*,807) (autose(7,kk),kk=1,4)
	write(*,808) (autose(8,kk),kk=1,4)

	write(*,721) (cor1se(1,kk),kk=1,9)
	write(*,722) (cor1se(2,kk),kk=1,9)
	write(*,723) (cor2se(1,kk),kk=1,9)
	write(*,724) (cor2se(2,kk),kk=1,9)
	write(*,725) (cor3se(1,kk),kk=1,9)

	write(55,701) ratse(1)
	write(55,702) ratse(2)
	write(55,703) ratse(3)
	write(55,704) ratse(4)
	write(55,705) ratse(5)
	write(55,706) ratse(6)
	write(55,707) ratse(7)
	write(55,708) ratse(8)

	write(55,801) (autose(1,kk),kk=1,4)
	write(55,802) (autose(2,kk),kk=1,4)
	write(55,803) (autose(3,kk),kk=1,4)
	write(55,804) (autose(4,kk),kk=1,4)
	write(55,805) (autose(5,kk),kk=1,4)
	write(55,806) (autose(6,kk),kk=1,4)
	write(55,807) (autose(7,kk),kk=1,4)
	write(55,808) (autose(8,kk),kk=1,4)

	write(55,721) (cor1se(1,kk),kk=1,9)
	write(55,722) (cor1se(2,kk),kk=1,9)
	write(55,723) (cor2se(1,kk),kk=1,9)
	write(55,724) (cor2se(2,kk),kk=1,9)
	write(55,725) (cor3se(1,kk),kk=1,9)

701     format('  output stan. dev.    ',2f14.4)
702     format('  employment/output    ',2f14.4) 
703     format(' destruction/employment',2f14.4)
704     format('    creation/employment',2f14.4)
705     format(' consumption/output    ',2f14.4)
706     format('  investment/output    ',2f14.4)
707     format('       solow/output    ',2f14.4)
708     format('output/productivity    ',2f14.4)

721     format('destruction & output     ',9f6.2)
722     format('   creation & output     ',9f6.2)
723     format('destruction & employment ',9f6.2)
724     format('   creation & employment ',9f6.2)
725     format('   creation & destruction',9f6.2)

801     format('      output',4f7.3)
802     format(' employement',4f7.3)
803     format(' destruction',4f7.3)
804     format('    creation',4f7.3)
805     format(' consumption',4f7.3)
806     format('  investment',4f7.3)
807     format('       solow',4f7.3)
808     format('productivity',4f7.3)
	end

***************************************************************************
*
*       AVESUB(); subroutine to calculate the average
*
***************************************************************************

	subroutine avesub()
	implicit real*8(a-h,o-z),integer(i-n)
	common/stat/ y(500,10),cor1(2,9),cor2(2,9),ave(10),stan(10),
     $  cor3(2,9),autocor(10,10),nobs,idump

	do 10 j = 1,8
	ave(j) = 0.
10      continue

	do 11 j = 1,8
	do 11 i = idump+1,idump+nobs
	ave(j) = ave(j) + y(i,j)
11      continue

	do 12 j = 1,8
	ave(j) = ave(j)/dble(nobs)
12      continue

	end

***************************************************************************
*
*       STANSUB(); subroutine to calculate the standard deviation
*
***************************************************************************

	subroutine stansub()
	implicit real*8(a-h,o-z),integer(i-n)
	common/stat/ y(500,10),cor1(2,9),cor2(2,9),ave(10),stan(10),
     $  cor3(2,9),autocor(10,10),nobs,idump

	do 10 j = 1,8
	stan(j) = 0.
10      continue

	do 11 j = 1,8
	do 11 i = idump+1,idump+nobs
	stan(j) = stan(j) + (y(i,j)-ave(j))**2.
11      continue

	do 12 j = 1,8
	stan(j) = (stan(j)/dble(nobs))**.5
12      continue

	end

***************************************************************************
*
*       CORSUB(); subroutine to calculate the correlation coefficient
*
***************************************************************************

	subroutine corsub()
	implicit real*8(a-h,o-z),integer(i-n)
	common/stat/ y(500,10),cor1(2,9),cor2(2,9),ave(10),stan(10),
     $  cor3(2,9),autocor(10,10),nobs,idump
	dimension cov1(2,9),cov2(2,9),cov3(1,9)


	do 10 k = 1,9
	cov3(1,k) = 0.0
	do 10 j = 1,2
	cov1(j,k) = 0.0
	cov2(j,k) = 0.0
10      continue

	
	do 15 k = 1,9
	if(k.le.5) then
	ibegin = idump+1+5-k
	iend   = idump+nobs
	else
	ibegin = idump+1
	iend   = idump+nobs+5-k
	endif

	do 11 i = ibegin,iend
	cov3(1,k) = cov3(1,k)
     $  + (y(i-5+k,4)-ave(4))*(y(i,3)-ave(3))
	do 11 j = 1,2
	cov1(j,k) = cov1(j,k)
     $  + (y(i-5+k,j+2)-ave(j+2))*(y(i,1)-ave(1))
	cov2(j,k) = cov2(j,k)                                  
     $  + (y(i-5+k,j+2)-ave(j+2))*(y(i,2)-ave(2))
11      continue
	
	cor3(1,k) = cov3(1,k)/dble(nobs-abs(k-5))
	cor3(1,k) = cor3(1,k)/(stan(3)*stan(4))
	
	do 12 j = 1,2
	cor1(j,k) = cov1(j,k)/dble(nobs-abs(k-5))
	cor2(j,k) = cov2(j,k)/dble(nobs-abs(k-5))
	cor1(j,k) = cor1(j,k)/(stan(j+2)*stan(1))
	cor2(j,k) = cor2(j,k)/(stan(j+2)*stan(2))
12      continue

15      continue
	

	end

***************************************************************************
*
*       AUTOSUB(); subroutine to calculate autocorrelation coefficients
*
***************************************************************************

	subroutine autosub()
	implicit real*8(a-h,o-z),integer(i-n)
	common/stat/ y(500,10),cor1(2,9),cor2(2,9),ave(10),stan(10),
     $  cor3(2,9),autocor(10,10),nobs,idump
	dimension autocov(10,10)


	do 10 k = 1,4
	do 10 j = 1,8
	autocov(j,k) = 0.
10      continue    

	do 11 k = 1,4
	do 11 j = 1,8
	do 12 i = idump+1+k,idump+nobs
	autocov(j,k) = autocov(j,k)
     $  + (y(i,j)-ave(j))*(y(i-k,j)-ave(j))
12      continue
	autocov(j,k) = autocov(j,k)/dble(nobs-k)
	autocor(j,k) = autocov(j,k)/(stan(j)**2.)
11      continue


	end

***************************************************************************
*
*       SCA(); subroutine to scale the state variable such that they are
*       between minus one and one
*
***************************************************************************

	real*8 function sca(x,i)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

	sca =  ( 2.*x -  stbar(i,2) - stbar(i,1) ) 
     $        /  (stbar(i,2)-stbar(i,1))

	end 

***************************************************************************
*
*       SCINV(); Inverse of SCA() 
*
***************************************************************************

	real*8 function scinv(x,i)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap
	scinv =  (    stbar(i,2) + stbar(i,1)  
     $        +    x*(stbar(i,2) - stbar(i,1))   ) / 2.

	end 

*
*
*        Define Chebyshev Polynomials
*
*

	real*8 function h(x,i)
	implicit real*8(a-h,o-z),integer(i-n)
	dimension a(200)

	a(1) = 1.
	a(2) = x

	if(i.le.1) then
	h =  a(i+1)
	goto 99
	endif

	
	if(i.gt.1) then

	do 10 j = 2,i
	a(j+1) = 2.*x*a(j) - a(j-1)
10      continue
	h =  a(i+1)

	endif

99      continue

	end


	real*8 function hh(s1,s2,s3,i1,i2,i3)
	implicit real*8(a-h,o-z),integer(i-n)
	
	hh = h(s1,i1)*h(s2,i2)*h(s3,i3)
	
	end

***************************************************************************
*
*       IPOINT(); Subroutine to allocate a number to a polynomial term
*
***************************************************************************

	integer function ipoint(i1,i2,i3)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

	ipoint = 1 + i2 
     $  + i1*(i2max+1) 
     $  + i3*(i1max+1)*(i2max+1)

c
c       i* goes from 0 to i*max
c
c       with this convention increasing i1max adds parameters at the
c       end of the parameter vector
c
	
	end

***************************************************************************
*
*       IPOINT(); Subroutine to allocate a number to a gridpoint
*
***************************************************************************

	integer function ispoint(is1,is2,is3)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap


	ispoint = is1 
     $  + (is2-1)*is1max 
     $  + (is3-1)*is1max*is2max

c
c       is* goes from 1 to is*max
c

	end

***************************************************************************
*
*       POL(); Function to calculate the parameterized expectation for
*       arbitrary values of the _scaled_ state variables
*
***************************************************************************

	real*8 function pol(s1,s2,s3,kk)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

	pol = 0.
	do 41 j1 = 1,i1max+1
	do 41 j2 = 1,i2max+1
	do 41 j3 = 1,i3max+1
	i1 = j1-1
	i2 = j2-1
	i3 = j3-1
	j = ipoint(i1,i2,i3)
	pol = pol + aold(j,kk)*hh(s1,s2,s3,i1,i2,i3)
41      continue

	pol = exp(pol)

	end


***************************************************************************
*
*       POLGRID(); Same as POL() but does it at the grid points for which
*       we have stored all the Chebyshev terms
*
***************************************************************************

	real*8 function polgrid(is1,is2,is3,kk)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

	jt = ispoint(is1,is2,is3)
	polgrid = 0.
	do 20 jj = 1,npar
	polgrid =  polgrid + aold(jj,kk)*cheb(jt,jj)
20      continue
	
	polgrid = exp(polgrid)

	end


***************************************************************************
*
*       RTBIS(); Nonlinear Equation Solver from Numerical Recipes
*
***************************************************************************


      REAL*8 FUNCTION rtbis(func,x1,x2,xacc)
      IMPLICIT REAL*8(A-H,O-Z),INTEGER(I-N)
      EXTERNAL func
      PARAMETER (JMAX=400)


60      continue

      fmid=func(x2)
      f=func(x1)

		if(f*fmid.ge.0.) then
		write(*,*) 'root must be bracketed in rtbis'
		stop
		endif

      if(f.lt.0.)then
	rtbis=x1
	dx=x2-x1
      else
	rtbis=x2                         
	dx=x1-x2
      endif

      do 11 j=1,JMAX
	dx=dx*.5
	xmid=rtbis+dx
	fmid=func(xmid)
	if(fmid.le.xacc) rtbis=xmid
c        if(ibaal.eq.1) write(*,'(3f14.6)') fmid,xmid,dx
c        if(abs(dx).lt.xacc .or. fmid.eq.0.) return
	
	IF(ABS(FMID).LT.XACC)RETURN
c
c
c       this is a slight adjustment of the numerical recipes subroutine
c       due to the numerical errors caused by the numerical integration
c       procedure there is the possibility that the function is not
c       continuous and that there is no zero. The n.r. subroutine 
c       would converge anyway. This algorithm would hit the bound
c       of maximum number of iterations
c
c
11    continue
      pause 'too many bisections in rtbis'
      END
C  (C) Copr. 1986-92 Numerical Recipes Software D.Q=.


***************************************************************************
*
*       FDENS(); Returns the density of a normal
*
***************************************************************************

	real*8 function fdens(x)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

	fdens = exp( -0.5*x**2./fstan**2.)/(fstan*(2.*pi)**0.5)

	return
	end

***************************************************************************
*
*       FNORMCUM();  CDF of a normal
*
***************************************************************************

	real*8 function fnormcum(x)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

	external fdens


	if(x.le.0.) then

		ff   = 0.
		yy   = (x+4.*fstan)/dble(nsimpson-1)
		do 1912 i = 1,nsimpson
		qx   = dble(i-1)*yy - 4.*fstan
		qw =  hsw(i)*yy*fdens(qx)
		ff = ff + qw
1912            continue
		fnormcum = ff

	else
		
		ff = 0.
		yy = (4.*fstan-x)/dble(nsimpson-1)
		do 1913 i = 1,nsimpson
		qx  =  dble(i-1)*yy + x
		qw  =  hsw(i)*yy*fdens(qx)
		ff  =  ff + qw
1913            continue
		fnormcum = 1.-ff


	endif

	return
	end


*************************************************************************
*
*       THE FOLLOWING TWO NUMERICAL RECIPES SUBROUTINES GENERATE RANDOM NUMBERS
*
*************************************************************************

	REAL*8 FUNCTION GASDEV(IDUM)

c       returns a normally distributed deviate with zero mean and unit 
c       variance, using RAN1(IDUM) as the source of uniform deviates

	IMPLICIT REAL*8(A-H,O-Z),INTEGER(I-N)
	save iset,gset
	DATA ISET/0/
	IF (ISET.EQ.0) THEN
1               V1=2.*RAN1(IDUM)-1.
		V2=2.*RAN1(IDUM)-1.
		R=V1**2.+V2**2.
		IF(R.GE.1..OR.R.EQ.0.)GO TO 1
		FAC=SQRT(-2.*LOG(R)/R)
		GSET=V1*FAC
		GASDEV=V2*FAC
		ISET=1
	ELSE
		GASDEV=GSET
		ISET=0
	ENDIF
	RETURN
	END



	REAL*8 FUNCTION RAN1(IDUM)
c
c       returns a uniform random deviae between 0.0 and 1.0.
c       set IDUM to any negative value to initialize or reinitialize
c       the sequence

	IMPLICIT REAL*8(A-H,O-Z),INTEGER(I-N)

	DIMENSION R(97)
	PARAMETER (M1=259200,IA1=7141,IC1=54773,RM1=1./M1)
	PARAMETER (M2=134456,IA2=8121,IC2=28411,RM2=1./M2)
	PARAMETER (M3=243000,IA3=4561,IC3=51349)
	save iff,ix1,ix2,ix3,r
	DATA IFF /0/
	IF(IDUM.LT.0.OR.IFF.EQ.0) THEN
		IFF = 1
		IX1 = MOD(IC1-IDUM,M1)
		IX1 = MOD(IA1*IX1+IC1,M1)
		IX2 = MOD(IX1,M2)
		IX1 = MOD(IA1*IX1+IC1,M1)
		IX3 = MOD(IX1,M3)
		DO 11 J=1,97
			IX1=MOD(IA1*IX1+IC1,M1)
			IX2=MOD(IA2*IX2+IC2,M2)
			R(J)=(DBLE(IX1)*DBLE(IX2)*RM2)*RM1
11              CONTINUE


		IDUM = 1

	ENDIF

	IX1=MOD(IA1*IX1+IC1,M1)
	IX2=MOD(IA2*IX2+IC2,M2)
	IX3=MOD(IA3*IX3+IC3,M3)
	J=1+(97*IX3)/M3
	IF(J.GT.97.OR.J.LT.1)PAUSE
	RAN1=R(J)
	R(J)=(DBLE(IX1)+DBLE(IX2)*RM2)*RM1
	RETURN
	END



***************************************************************************
*
*   HPFILT();
*   Kalman smoothing routine for HP filter written by E Prescott.
*   y=data series, d=deviations from trend, t=trend, n=no. obs,
*   s=smoothing parameter (eg, 1600 for std HP).
*   Array v is scratch area and must have dimension at least 3n.
*   If IOPT=1 and n and s are the same as for the previous call,
*   the numbers in v are not recomputed.  This reduces execution
*   time by about a third.  Note that if this option is exercised,
*   v cannot be used for other purposes between calls.
*
***************************************************************************

      subroutine hpfilt(y,t,d,v,n,nw,s,iopt)
      implicit real*8(a-h,o-z),integer(i-n)
      real*8 y(n),t(n),v(n,nw),d(n)
      double precision m1,m2,v11,v12,v22,x,z,b11,b12,b22,det,e1,e2
      data ss,nn/0.0,0/


c
c     compute sequences of covariance matrix for f[x(t),x(t-1) | y(<t)]
c
      if(iopt.ne.1.or.nn.ne.n.or.s.ne.ss)  then
	ss=s
	nn=n
	v11=1.d0
	v22=1.d0
	v12=0.d0
       do 5 i=3,n
	x=v11
	z=v12
	v11=1.d0/s + 4.d0*(x-z) + v22
	v12=2.d0*x - z
	v22=x
	det=v11*v22-v12*v12
	v(i,1)=v22/det
	v(i,3)=v11/det
	v(i,2)=-v12/det
	x=v11+1.d0
	z=v11
	v11=v11-v11*v11/x
	v22=v22-v12*v12/x
	v12=v12-z*v12/x
  5    continue
				       endif
c
c     this is the forward pass
c
      m1=y(2)
      m2=y(1)
      do 10 i=3,n
	x=m1
	m1=2.d0*m1-m2
	m2=x
	t(i-1)= v(i,1)*m1+v(i,2)*m2
	d(i-1)= v(i,2)*m1+v(i,3)*m2
	det=v(i,1)*v(i,3)-v(i,2)*v(i,2)
	v11=v(i,3)/det
	v12=-v(i,2)/det
	z=(y(i)-m1)/(v11+1.d0)
	m1=m1+v11*z
	m2=m2+v12*z
 10     continue
      t(n)=m1
      t(n-1)=m2
c
c       this is the backward pass
c
	 m1=y(n-1)
	 m2=y(n)
	do 15 i=n-2,1,-1
	   i1=i+1
	   ib=n-i+1
	 x=m1
	 m1=2.d0*m1 - m2
	 m2=x
c
c           combine info for y(.lt.i) with info for y(.ge.i)
c
	 if(i.gt.2)                 then
	   e1=v(ib,3)*m2 + v(ib,2)*m1 + t(i)
	   e2=v(ib,2)*m2 + v(ib,1)*m1 + d(i)
	   b11=v(ib,3)+v(i1,1)
	   b12=v(ib,2)+v(i1,2)
	   b22=v(ib,1)+v(i1,3)
	   det=b11*b22-b12*b12
	   t(i)=(-b12*e1+b11*e2)/det
				    endif
c
c           end of combining
c
	det=v(ib,1)*v(ib,3)-v(ib,2)*v(ib,2)
	v11=v(ib,3)/det
	v12=-v(ib,2)/det
	z=(y(i)-m1)/(v11+1.d0)
	 m1=m1+v11*z
	 m2=m2+v12*z
 15     continue
       t(1)=m1
       t(2)=m2
	do 20 i=1,n
 20      d(i)=y(i)-t(i)
	return
	end


***************************************************************************
*
*       POLTRUNC(); Subroutine to calculate the amount of endogenous
*       separation and the rental rate for a given level of the three
*       state variables
*
***************************************************************************

	subroutine poltrunc()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap


	external fdens,equil,equilcap

	if(icap.eq.1) then

c       Calculate the rental price of capital.  This program assumes
c       that the firms observe the aggregate shock, the, but not
c       the firm specific shock in making the capital decision.
c       Firms maximize the expected surplus which is equal to
c       expected profits plus a constant.
c
c       Note that per capita consumption is not correlated with the
c       idiosyncratic shock. Because of this we don't have to
c       worry about marginal utility of consumption anywhere.
c
c       This boils down to the firm setting the rental rate equal
c       to the expected marginal product of capital _conditional_
c       on not breaking up.  Unfortunately, the truncation value
c       at which firms break up depends on the interest rate,
c       so we need a non-linear equation solver to find a level
c       of the rental rate and the truncation parameter that are
c       consistent.
c
c       Note that this part is not affected by the first wave of
c       exogenous break-ups (auto1) which takes place after the
c       capital decision has been made but before production.
c

c
c       get initial guess assuming there are no endogenous break-ups
c

	rental = alpha*the*asum*(cap/pemployed)**(alpha-1.)

c
c       calculate the cut-off level for the idiosyncratic shock
c       where ic problems break the relation up
c

	rentemp = auto2*gg+(1.-auto2)*ww-ww-xvalue-bvalue
	temp    = rentemp-rental*(cap/pemployed)

	trunc    = -temp/(the*(cap/pemployed)**alpha)

	if(trunc.le.exp(-4.*fstan)) trunc = exp(-4.*fstan)
	trunclog = log(trunc)
c        truncpr  = 1.-fnormcum(trunclog)

	
	x1 = trunclog-0.01
	x2 = trunclog+2.*fstan
	xacc = 0.00000001
	trunclog  = rtbis(equil,x1,x2,xacc)
c        trunc     = exp(trunclog)
	truncpr   = 1.-fnormcum(trunclog)

c        write(*,'(4f14.6)') rental,trunc,trunclog,truncpr                

	else

	rental = alpha*the*asum*(cap/(auto1*pemployed))**(alpha-1.)

	x1 = rental*0.8
	x2 = rental*1.2
	
	xacc = 0.00000001
	rental = rtbis(equilcap,x1,x2,xacc)
	
	endif
	
	end

***************************************************************************
*
*       POLPROD(); Subroutine to calculate production and the matching
*       probabilities
*
***************************************************************************

	subroutine polprod()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

	external fdens,equil,equilcap

	prod  = 0.

	yy = (4.*fstan-trunclog)/dble(nsimpson-1)
	do  15 i = 1,nsimpson
	qx =  dble(i-1)*yy + trunclog
	qw =  hsw(i)*yy*fdens(qx)
	qx =  exp(qx)
	capi =     dble(icap)*(cap/pemployed)
     $       + (1.-dble(icap))
     $       * (the*alpha*qx/rental)**(-1./(alpha-1.))

	prodi = the*qx*capi**alpha
	prod = prod + prodi*qw*pemployed*auto1  
15      continue
	

*
*       calculate next period's capital stock
*
	
	cons1   = exp1**(1./gam)-(1.-auto1*truncpr*pemployed)*bvalue

*
*       calculate next period's fraction of people employed
*
	unemp = 1.-auto1*auto2*pemployed*truncpr
	probf = eta/wf
	if(probf.gt.1) then
	vac = 0.
	pmatch = 0.
	vacpr = 0.
	else
	vac = ( (prob*unemp/probf)**abba - unemp**abba )**(1./abba)
	pmatch =  prob*vac/
     $  (vac**abba+unemp**abba)**(1./abba)
	vacpr =   pmatch*unemp/vac
	endif

	capnew  = prod + (1.-depr)*cap - cons1   -   vac*eta
     
	pempnew = 1.-unemp + pmatch*unemp

	end

***************************************************************************
*
*       POLEXP(); Subroutine to calculate the four conditional expectations
*       conditional on this period's information. 
*       Inputs for this subroutine are the capital choice made this period,
*       cap, the fraction of matches at the beginning of the next period,
*       pemployed, this period's aggregate technology shock, theold,
*       this period's marginal utility of consumption, and the matching
*       probability, pmatch.
*
*       The four conditional expectations are:
*
*       1. The conditional expectations from the household's savings
*          decision.
*       2. The expected value of future benefits for the match when
*          you leave this period in a match.
*       3. The expected value of future benefits for the worker when
*          he enters the matching pool this period.
*       4. The expected value of future benefits for the firm when
*          he enters the matching pool this period.
*
*

	subroutine polexp()
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

	external equil,fdens,equilcap

	expcons  = 0.
	expmatch = 0.
	expwork  = 0.
	expfirm  = 0.

	s1new  = sca(cap,1)
	s3new  = sca(pemployed,3)        
	
	do 30 jnew = 1,nhermite
		
	the   = ro*log(theold)+hhx(jnew)*sig*(2.**0.5)
	s2new  = sca(the,2)
	the   = exp(the)        


	gg  = pol(s1new,s2new,s3new,2)
	ww  = pol(s1new,s2new,s3new,3)
	wf  = pol(s1new,s2new,s3new,4)
	exp1 = pol(s1new,s2new,s3new,1)

	call poltrunc()

	rentadj = dble(icap)*truncpr*auto1 + (1.-dble(icap))*1.
	expcons = expcons +  hhw(jnew)*bta*
     $  (rentadj*rental + 1.-depr)*exp1

c
c       note that the firms who break up, either endogenously or
c       exogenously before the production phase, do not pay
c       rental costs
c

	yy = (4.*fstan-trunclog)/dble(nsimpson-1)

	aa = 0.
	bb = 0.
	cc = 0.

	do  35 knew = 1,nsimpson
	qx =  dble(knew-1)*yy + trunclog
	qw =  hsw(knew)*yy*fdens(qx)
	qx =  exp(qx)
	
	capi =     dble(icap)*(cap/pemployed)
     $       + (1.-dble(icap))
     $       * (the*alpha*qx/rental)**(-1./(alpha-1.))
	aasurplus =  the*qx*capi**alpha-rental*capi 
	aa        =  aa + qw*aasurplus
	bbtemp    =  (1.-share)*(the*qx*capi**alpha-rental*capi
     $               +auto2*gg+(1.-auto2)*ww-bvalue-ww)
	bb        =  bb + qw*bbtemp
	cctemp    =  share*(the*qx*capi**alpha-rental*capi
     $               +auto2*gg+(1.-auto2)*ww-bvalue-ww)
	cc        =  cc + qw*cctemp
35      continue

	aa = aa*auto1
     $     +      auto2*auto1*truncpr*gg
     $     + (1.-auto2)*auto1*truncpr*ww
     $     + (1.-truncpr*auto1)*(bvalue+ww)

	bb = pmatch*(
     $                   bb*auto1 + bvalue + ww
     $               )
     $     + (1.-pmatch)*(bvalue+ww)

	cc = cc*auto1

	expmatch = expmatch + aa*bta*hhw(jnew)*(exp1/expold)
	expwork  = expwork  + bb*bta*hhw(jnew)*(exp1/expold)
	expfirm  = expfirm  + cc*bta*hhw(jnew)*(exp1/expold)
30      continue

	expcons  = expcons/(pi**0.5)       
	expmatch = expmatch/(pi**0.5)
	expwork  = expwork/(pi**0.5)
	expfirm  = expfirm/(pi**0.5)

	end

***************************************************************************
*
*       EQUIL: Function that calculates for a given level of the truncation
*       level the corresponding interest rate and the truncation
*       level that corresponds with this interest rate. (With rtbis we
*       find the level such that the input value equals the output value).
*
***************************************************************************

	real*8 function equil(trlog)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

	asum2  = 0.

	yy = (4.*fstan-trlog)/dble(nsimpson-1)
	do  15 i = 1,nsimpson
	qx =  dble(i-1)*yy + trlog
	qw =  hsw(i)*yy*fdens(qx)
	asum2 = asum2 + qw*exp(qx)      
15      continue

	trpr   = 1.-fnormcum(trlog)
	asum2  = asum2/trpr
	rental = alpha*the*asum2*(cap/pemployed)**(alpha-1.)

c
c       calculate the cut-off level for the idiosyncratic shock
c       where ic problems break the relation up
c

	temp = rentemp-rental*(cap/pemployed)

	trunc    = -temp/(the*(cap/pemployed)**alpha)

	if(trunc.le.exp(-4.*fstan)) trunc = exp(-4.*fstan)
	trunclog = log(trunc)

	equil = trunclog - trlog

c        write(*,'(4f14.6)') rental,trunclog,trlog,equil

	return
	end

***************************************************************************
*
*       EQUILCAP: Function to calculate the equilibrium interest rate
*
***************************************************************************

	real*8 function equilcap(rr)
	implicit real*8(a-h,o-z),integer(i-n)
	parameter(npar=27)
	common/c1/   stu(1000,10),stbar(10,2),st(1000,10),
     $  cheb(4000,64),xx(npar,npar),hhx(1000),hhw(1000),hsw(1000),
     $  aold(npar,4),anew(npar,4),
     $  alpha,bta,depr,gam,ro,sig,pi,asum,xvalue,fstan,share,
     $  prob,eta,auto1,auto2,abba,bvalue,xacc,acc,
     $  cap,the,pemployed,capnew,theold,pempnew,prod,cons1,unemp,
     $  rentemp,rental,pmatch,vacpr,vac,trunc,trunclog,truncpr,
     $  exp1,expold,expcons,expmatch,expwork,expfirm,gg,ww,wf
	common/i1/ i1max,i2max,i3max,is1max,is2max,is3max,
     $  nhermite,nsimpson,icreate,icap

c
c       calculate the cut-off level for the idiosyncratic shock
c


	if(xvalue+bvalue+ww-gg.lt.0) then
	trunc    = exp(-4.*fstan)
	trunclog =     -4.*fstan
	truncpr  = 1.
	goto 66
	endif


	a1 = the*(rr/(alpha*the))**(alpha/(alpha-1.))

	a2 = rr*(rr/(alpha*the))**(1./(alpha-1.))

	trunc =  ( (xvalue+bvalue+ww-gg)/(a1-a2) )**(1.-alpha)

	if(trunc.lt.exp(-4.*fstan)) then
	trunc     = exp(-4.*fstan)
	trunclog  = -4.*fstan
	truncpr   =  1.
	else
	trunclog = log(trunc)
	truncpr  = 1.-fnormcum(trunclog)
	endif

	if(trunc.gt.exp( 4.*fstan)) then
	write(*,*) ' a low probability event has happened'
	write(*,*) ' adjust bounds in subroutine '
	stop
	endif

66      continue

	capsum = 0.
	
	hh = (4.*fstan-trunclog)/dble(nsimpson-1)

	do 100 i = 1,nsimpson

	qx = dble(i-1)*hh + trunclog
	qw = hsw(i)*hh*fdens(qx)
	qx = exp(qx)
	capi = (the*alpha*qx/rr)**(-1./(alpha-1.))
	capsum = capsum + capi*qw*pemployed*auto1
100     continue

	equilcap = cap - capsum

c       write(*,*) rr,truncpr,equilcap
	 
	return
	end


### drwinput.dat (this file contains all the necessary input files)

In [ ]:
*
*
*	drw.dat
*
*

   2    2     2      3     3     3      5       15     0.36
  0.99 0.025  -1.    0.95    0.007   0.     0.4      0.3
  20.0  32.0    0.7  1.      .101461   .0001 .0000001 1000  3  2.220305
  0.5   1.  .203072   1.  1.27  0.932  
  1      0    0    0    1     1      0     0

i1max i2max i3max is1max is2max is3max nhermite nlegendre alpha 
 beta  depr  gam   ro     sig   xvalue  pmatch    eps
  bl1 bh1    bl3   bh3    fstan  acc   xacc    itotal  i2bound bvalue
  share prob eta  auto2    abba   auto1
  istst acc1 acc2 acc3 imom icreate impuls icap

npar = 27  complete basis                                                       


*
*
*	drwcoef.dat
*
*

  -9.200183269703941E-001   2.962235375747067E-002  -3.006985422786386E-003
  -7.799155734392171E-002  -1.550120303228971E-002  -7.295920775553085E-004
  -6.792677066396072E-004   1.103446304752051E-004   2.875642390224560E-005
   3.574283653621123E-004  -1.064932320770129E-004  -6.156883021499502E-005
  -1.929467182495962E-004  -2.946841889044803E-004  -1.003189611458242E-004
  -8.476780750557353E-005  -8.544308275185718E-005   7.710340560980911E-006
   3.943643555301271E-005   2.510056565521542E-005   1.274847266180390E-005
   2.926680767829402E-005   4.726739651522419E-005  -1.319350400695840E-005
   1.021850200372219E-005  -1.597146345455923E-005   2.968080973029142E-006
        5.405016061058974  -3.396029352475934E-002   3.338836431529802E-003
   6.074603399561593E-002   1.685275151693107E-002   9.509882515248478E-004
   1.590340991019696E-003   1.044771799152450E-004  -1.819754401253772E-005
  -7.518501456267414E-004   2.060087900980791E-005   6.373815693291312E-005
   1.213365651333659E-004   3.152090181330377E-004   1.180590362724684E-004
   9.877922616409098E-005   1.039275851080990E-004  -8.988537755285664E-006
  -2.480225542073479E-005  -2.201170198923001E-005  -1.504934253743380E-005
  -2.756816447783179E-005  -5.599252308396123E-005   1.539456141807582E-005
  -1.296671736164316E-005   1.820480633618575E-005  -4.134753673532000E-006
        5.403073862528357  -3.414390640394351E-002   3.324098588214430E-003
   6.072196869034455E-002   1.679533647495255E-002   9.397308008413667E-004
   1.573310907083038E-003   9.645833110035979E-005  -1.563163823747407E-005
  -7.044153521748329E-004   3.800412535475900E-005   6.661775294149561E-005
   1.413677647824871E-004   3.198712227684064E-004   1.086014691574529E-004
   9.810017202057656E-005   9.468944087405497E-005  -5.475616035662968E-006
  -2.544069271045533E-005  -2.542083785918734E-005  -1.219209351722362E-005
  -3.041233954943849E-005  -4.745782211733553E-005   1.086131627975853E-005
  -1.029028889292715E-005   1.385296035152930E-005  -1.482264353071060E-006
       -1.359509618609050   1.458268961438060E-001   1.169733877832986E-002
   1.785027968924941E-001   6.005970224650448E-002   2.503483026817768E-003
   8.794001450418931E-003  -7.608905485164228E-004  -1.826627851887316E-003
  -6.366106843874403E-002  -3.044721986512771E-003   1.814936508578634E-003
  -5.358852943830096E-003   9.682739841123417E-003   2.889656388947184E-003
   3.420069352862577E-003   2.688711705668260E-003  -6.906722702129753E-004
   4.965609612785814E-004  -4.106152903748044E-005  -6.316061541969629E-004
  -2.545809300658943E-004  -2.292411686571850E-003   1.052686772959878E-003
  -6.463931996715418E-004   1.085690582878403E-003  -3.874328346897032E-004
   

  
*
*
*	cheb0.dat
*
*
                                             
 -.7071067811865  .7071067811865  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.8660254037844  .0000000000000  .8660254037844  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9238795325113 -.3826834323651  .3826834323651  .9238795325113  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9510565162952 -.5877852522925  .0000000000000  .5877852522925  .9510565162952
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9659258262891 -.7071067811865 -.2588190451025  .2588190451025  .7071067811865
  .9659258262891  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9749279121818 -.7818314824680 -.4338837391176  .0000000000000  .4338837391176
  .7818314824680  .9749279121818  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9807852804032 -.8314696123025 -.5555702330196 -.1950903220161  .1950903220161
  .5555702330196  .8314696123025  .9807852804032  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9848077530122 -.8660254037844 -.6427876096865 -.3420201433257  .0000000000000
  .3420201433257  .6427876096865  .8660254037844  .9848077530122  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9876883405951 -.8910065241884 -.7071067811865 -.4539904997395 -.1564344650402
  .1564344650402  .4539904997395  .7071067811865  .8910065241884  .9876883405951
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9898214418809 -.9096319953545 -.7557495743543 -.5406408174556 -.2817325568414
  .0000000000000  .2817325568414  .5406408174556  .7557495743543  .9096319953545
  .9898214418809  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9914448613738 -.9238795325113 -.7933533402912 -.6087614290087 -.3826834323651
 -.1305261922201  .1305261922201  .3826834323651  .6087614290087  .7933533402912
  .9238795325113  .9914448613738  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9927088740981 -.9350162426854 -.8229838658937 -.6631226582408 -.4647231720438
 -.2393156642876  .0000000000000  .2393156642876  .4647231720438  .6631226582408
  .8229838658937  .9350162426854  .9927088740981  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9937122098932 -.9438833303084 -.8467241992283 -.7071067811865 -.5320320765153
 -.3302790619552 -.1119644761033  .1119644761033  .3302790619552  .5320320765153
  .7071067811865  .8467241992283  .9438833303084  .9937122098932  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9945218953683 -.9510565162952 -.8660254037844 -.7431448254774 -.5877852522925
 -.4067366430758 -.2079116908178  .0000000000000  .2079116908178  .4067366430758
  .5877852522925  .7431448254774  .8660254037844  .9510565162952  .9945218953683
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9951847266722 -.9569403357322 -.8819212643484 -.7730104533627 -.6343932841636
 -.4713967368260 -.2902846772545 -.0980171403296  .0980171403296  .2902846772545
  .4713967368260  .6343932841636  .7730104533627  .8819212643484  .9569403357322
  .9951847266722  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9957341762950 -.9618256431728 -.8951632913551 -.7980172272802 -.6736956436466
 -.5264321628774 -.3612416661872 -.1837495178166  .0000000000000  .1837495178166
  .3612416661872  .5264321628774  .6736956436466  .7980172272802  .8951632913551
  .9618256431728  .9957341762950  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9961946980917 -.9659258262891 -.9063077870367 -.8191520442890 -.7071067811865
 -.5735764363510 -.4226182617407 -.2588190451025 -.0871557427477  .0871557427477
  .2588190451025  .4226182617407  .5735764363510  .7071067811865  .8191520442890
  .9063077870366  .9659258262891  .9961946980917  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9965844930067 -.9694002659393 -.9157733266551 -.8371664782625 -.7357239106731
 -.6142127126897 -.4759473930371 -.3246994692047 -.1645945902807  .0000000000000
  .1645945902807  .3246994692047  .4759473930371  .6142127126897  .7357239106731
  .8371664782625  .9157733266551  .9694002659393  .9965844930067  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9969173337331 -.9723699203977 -.9238795325113 -.8526401643541 -.7604059656000
 -.6494480483302 -.5224985647159 -.3826834323651 -.2334453638559 -.0784590957278
  .0784590957278  .2334453638559  .3826834323651  .5224985647159  .6494480483302
  .7604059656000  .8526401643541  .9238795325113  .9723699203977  .9969173337331
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9972037971812 -.9749279121818 -.9308737486442 -.8660254037844 -.7818314824680
 -.6801727377709 -.5633200580636 -.4338837391176 -.2947551744109 -.1490422661762
  .0000000000000  .1490422661762  .2947551744109  .4338837391176  .5633200580636
  .6801727377709  .7818314824680  .8660254037844  .9308737486442  .9749279121818
  .9972037971812  .0000000000000  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9974521146103 -.9771468659712 -.9369497249998 -.8776789895673 -.8005412409244
 -.7071067811865 -.5992776665113 -.4792489867201 -.3494641795991 -.2125652895530
 -.0713391831992  .0713391831992  .2125652895530  .3494641795991  .4792489867201
  .5992776665113  .7071067811865  .8005412409244  .8776789895673  .9369497249998
  .9771468659712  .9974521146103  .0000000000000  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9976687691905 -.9790840876823 -.9422609221188 -.8878852184024 -.8169698930104
 -.7308359642781 -.6310879443261 -.5195839500354 -.3984010898462 -.2697967711570
 -.1361666490962  .0000000000000  .1361666490962  .2697967711570  .3984010898462
  .5195839500354  .6310879443261  .7308359642781  .8169698930104  .8878852184024
  .9422609221188  .9790840876823  .9976687691905  .0000000000000  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9978589232386 -.9807852804032 -.9469301294951 -.8968727415327 -.8314696123025
 -.7518398074790 -.6593458151001 -.5555702330196 -.4422886902190 -.3214394653032
 -.1950903220161 -.0654031292301  .0654031292301  .1950903220161  .3214394653032
  .4422886902190  .5555702330196  .6593458151001  .7518398074790  .8314696123025
  .8968727415327  .9469301294951  .9807852804032  .9978589232386  .0000000000000
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9980267284283 -.9822872507287 -.9510565162952 -.9048270524660 -.8443279255020
 -.7705132427758 -.6845471059287 -.5877852522925 -.4817536741017 -.3681245526847
 -.2486898871649 -.1253332335643  .0000000000000  .1253332335643  .2486898871649
  .3681245526847  .4817536741017  .5877852522925  .6845471059287  .7705132427758
  .8443279255020  .9048270524660  .9510565162952  .9822872507287  .9980267284283
  .0000000000000  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9981755542233 -.9836199069471 -.9547208665085 -.9118998459921 -.8557812723014
 -.7871834806091 -.7071067811865 -.6167188726285 -.5173378141777 -.4104128054528
 -.2975030538552 -.1802550378139 -.0603784974223  .0603784974223  .1802550378139
  .2975030538552  .4104128054528  .5173378141777  .6167188726285  .7071067811865
  .7871834806091  .8557812723014  .9118998459921  .9547208665085  .9836199069471
  .9981755542233  .0000000000000  .0000000000000  .0000000000000  .0000000000000
 -.9983081582713 -.9848077530122 -.9579895123155 -.9182161068803 -.8660254037844
 -.8021231927550 -.7273736415730 -.6427876096865 -.5495089780708 -.4487991802005
 -.3420201433257 -.2306158707424 -.1160929141252  .0000000000000  .1160929141252
  .2306158707424  .3420201433257  .4487991802005  .5495089780708  .6427876096865
  .7273736415730  .8021231927550  .8660254037844  .9182161068803  .9579895123155
  .9848077530122  .9983081582713  .0000000000000  .0000000000000  .0000000000000
 -.9984268150178 -.9858710185182 -.9609173219451 -.9238795325113 -.8752234219088
 -.8155608689593 -.7456421648832 -.6663465779520 -.5786712961798 -.4837188871052
 -.3826834323651 -.2768355114248 -.1675062233047 -.0560704472372  .0560704472372
  .1675062233047  .2768355114248  .3826834323651  .4837188871052  .5786712961798
  .6663465779520  .7456421648832  .8155608689593  .8752234219088  .9238795325113
  .9609173219451  .9858710185182  .9984268150178  .0000000000000  .0000000000000
 -.9985334138511 -.9868265225415 -.9635499925192 -.9289767198168 -.8835120444460
 -.8276889981569 -.7621620551276 -.6876994588534 -.6051742151938 -.5155538571770
 -.4198891015603 -.3193015301360 -.2149704402110 -.1081190184239  .0000000000000
  .1081190184239  .2149704402110  .3193015301360  .4198891015603  .5155538571770
  .6051742151938  .6876994588534  .7621620551276  .8276889981569  .8835120444460
  .9289767198168  .9635499925192  .9868265225415  .9985334138511  .0000000000000
 -.9986295347546 -.9876883405951 -.9659258262891 -.9335804264972 -.8910065241884
 -.8386705679454 -.7771459614570 -.7071067811865 -.6293203910498 -.5446390350150
 -.4539904997395 -.3583679495453 -.2588190451025 -.1564344650402 -.0523359562429
  .0523359562429  .1564344650402  .2588190451025  .3583679495453  .4539904997395
  .5446390350150  .6293203910498  .7071067811865  .7771459614570  .8386705679454
  .8910065241884  .9335804264972  .9659258262891  .9876883405951  .9986295347546
*
*
*	herm.pro
*
*

    .01995324    .39361932    .94530872    .39361932    .01995324    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00453001    .15706732    .72462960    .72462960    .15706732    .00453001
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00097178    .05451558    .42560725    .81026462    .42560725    .05451558
    .00097178    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00019960    .01707798    .20780233    .66114701    .66114701    .20780233
    .01707798    .00019960    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00003961    .00494362    .08847453    .43265156    .72023522    .43265156
    .08847453    .00494362    .00003961    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000764    .00134365    .03387439    .24013861    .61086263    .61086263
    .24013861    .03387439    .00134365    .00000764    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000144    .00034682    .01191140    .11722788    .42935975    .65475929
    .42935975    .11722788    .01191140    .00034682    .00000144    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000027    .00008574    .00390539    .05160799    .26049231    .57013524
    .57013524    .26049231    .05160799    .00390539    .00008574    .00000027
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000005    .00002043    .00120746    .02086278    .14032332    .42161630
    .60439319    .42161630    .14032332    .02086278    .00120746    .00002043
    .00000005    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000001    .00000472    .00035509    .00785005    .06850553    .27310561
    .53640591    .53640591    .27310561    .06850553    .00785005    .00035509
    .00000472    .00000001    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000106    .00010000    .00277807    .03078003    .15848892
    .41202869    .56410031    .41202869    .15848892    .03078003    .00277807
    .00010000    .00000106    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000023    .00002712    .00093228    .01288031    .08381004
    .28064746    .50792948    .50792948    .28064746    .08381004    .01288031
    .00093228    .00002712    .00000023    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000005    .00000711    .00029864    .00506735    .04092003
    .17264830    .40182647    .53091794    .40182647    .17264830    .04092003
    .00506735    .00029864    .00000711    .00000005    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000001    .00000181    .00009181    .00188852    .01864004
    .09730175    .28480729    .48349569    .48349569    .28480729    .09730175
    .01864004    .00188852    .00009181    .00000181    .00000001    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000045    .00002721    .00067088    .00798887
    .05081039    .18363270    .39160899    .50297489    .39160899    .18363270
    .05081039    .00798887    .00067088    .00002721    .00000045    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000011    .00000780    .00022834    .00324377
    .02481052    .10901721    .28667551    .46224367    .46224367    .28667551
    .10901721    .02481052    .00324377    .00022834    .00000780    .00000011
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000003    .00000217    .00007478    .00125498
    .01141407    .06017965    .19212032    .38166907    .47902370    .38166907
    .19212032    .06017965    .01141407    .00125498    .00007478    .00000217
    .00000003    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000001    .00000059    .00002366    .00046489
    .00497840    .03114037    .11910236    .28697143    .44354523    .44354523
    .28697143    .11910236    .03114037    .00497840    .00046489    .00002366
    .00000059    .00000001    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000016    .00000725    .00016556
    .00206957    .01520708    .06889029    .19864490    .37214382    .45819659
    .37214382    .19864490    .06889029    .01520708    .00206957    .00016556
    .00000725    .00000016    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000004    .00000216    .00005689
    .00082369    .00704836    .03744547    .12773962    .28617954    .42693116
    .42693116    .28617954    .12773962    .03744547    .00704836    .00082369
    .00005689    .00000216    .00000004    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000001    .00000063    .00001892
    .00031508    .00311571    .01924310    .07688900    .20362114    .36308899
    .43986872    .36308899    .20362114    .07688900    .01924310    .00311571
    .00031508    .00001892    .00000063    .00000001    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000018    .00000610
    .00011623    .00131906    .00939790    .04359823    .13511333    .28463224
    .41204365    .41204365    .28463224    .13511333    .04359823    .00939790
    .00131906    .00011623    .00000610    .00000018    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000005    .00000192
    .00004147    .00053677    .00438128    .02341593    .08417308    .20737048
    .35451730    .42357729    .35451730    .20737048    .08417308    .02341593
    .00438128    .00053677    .00004147    .00000192    .00000005    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000001    .00000059
    .00001435    .00021062    .00195733    .01196842    .04951489    .14139461
    .28256139    .39860472    .39860472    .28256139    .14139461    .04951489
    .01196842    .00195733    .00021062    .00001435    .00000059    .00000001
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000017
    .00000482    .00007991    .00084079    .00584550    .02763966    .09076884
    .21014269    .34641894    .40897117    .34641894    .21014269    .09076884
    .02763966    .00584550    .00084079    .00007991    .00000482    .00000017
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000005
    .00000158    .00002939    .00034831    .00273792    .01470383    .05514418
    .14673585    .28013093    .38639489    .38639489    .28013093    .14673585
    .05514418    .01470383    .00273792    .00034831    .00002939    .00000158
    .00000005    .00000000    .00000000    .00000000    .00000000    .00000000

*
*
*	herm.dat
*
*

  -2.02018287   -.95857246    .00000000    .95857246   2.02018287    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -2.35060497  -1.33584907   -.43607741    .43607741   1.33584907   2.35060497
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -2.65196136  -1.67355163   -.81628788    .00000000    .81628788   1.67355163
   2.65196136    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -2.93063742  -1.98165676  -1.15719371   -.38118699    .38118699   1.15719371
   1.98165676   2.93063742    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -3.19099320  -2.26658058  -1.46855329   -.72355102    .00000000    .72355102
   1.46855329   2.26658058   3.19099320    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -3.43615912  -2.53273167  -1.75668365  -1.03661083   -.34290133    .34290133
   1.03661083   1.75668365   2.53273167   3.43615912    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -3.66847085  -2.78329010  -2.02594802  -1.32655708   -.65680957    .00000000
    .65680957   1.32655708   2.02594802   2.78329010   3.66847085    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -3.88972490  -3.02063703  -2.27950708  -1.59768264   -.94778839   -.31424038
    .31424038    .94778839   1.59768264   2.27950708   3.02063703   3.88972490
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -4.10133760  -3.24660898  -2.51973569  -1.85310765  -1.22005504   -.60576388
    .00000000    .60576388   1.22005504   1.85310765   2.51973569   3.24660898
   4.10133760    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -4.30444857  -3.46265693  -2.74847072  -2.09518326  -1.47668273   -.87871379
   -.29174551    .29174551    .87871379   1.47668273   2.09518326   2.74847072
   3.46265693   4.30444857    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -4.49999071  -3.66995037  -2.96716693  -2.32573249  -1.71999258  -1.13611559
   -.56506958    .00000000    .56506958   1.13611559   1.71999258   2.32573249
   2.96716693   3.66995037   4.49999071    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -4.68873894  -3.86944790  -3.17699916  -2.54620216  -1.95178799  -1.38025854
   -.82295145   -.27348105    .27348105    .82295145   1.38025854   1.95178799
   2.54620216   3.17699916   3.86944790   4.68873894    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -4.87134519  -4.06194668  -3.37893209  -2.75776292  -2.17350283  -1.61292431
  -1.06764873   -.53163300    .00000000    .53163300   1.06764873   1.61292431
   2.17350283   2.75776292   3.37893209   4.06194668   4.87134519    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -5.04836401  -4.24811787  -3.57376907  -2.96137751  -2.38629909  -1.83553160
  -1.30092086   -.77668292   -.25826775    .25826775    .77668292   1.30092086
   1.83553160   2.38629909   2.96137751   3.57376907   4.24811787   5.04836401
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -5.22027169  -4.42853281  -3.76218735  -3.15784882  -2.59113379  -2.04923171
  -1.52417062  -1.01036839   -.50352016    .00000000    .50352016   1.01036839
   1.52417062   2.04923171   2.59113379   3.15784882   3.76218735   4.42853281
   5.22027169    .00000000    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -5.38748089  -4.60368245  -3.94476404  -3.34785457  -2.78880606  -2.25497400
  -1.73853771  -1.23407622   -.73747373   -.24534071    .24534071    .73747373
   1.23407622   1.73853771   2.25497400   2.78880606   3.34785457   3.94476404
   4.60368245   5.38748089    .00000000    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -5.55035187  -4.77399234  -4.12199555  -3.53197288  -2.97999121  -2.45355212
  -1.94496295  -1.44893425   -.96149963   -.47945071    .00000000    .47945071
    .96149963   1.44893425   1.94496295   2.45355212   2.97999121   3.53197288
   4.12199555   4.77399234   5.55035187    .00000000    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -5.70920135  -4.93983413  -4.29431248  -3.71070153  -3.16526591  -2.64563744
  -2.14423359  -1.65587437  -1.17671396   -.70368610   -.23417914    .23417914
    .70368610   1.17671396   1.65587437   2.14423359   2.64563744   3.16526591
   3.71070153   4.29431248   4.93983413   5.70920135    .00000000    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -5.86430950  -5.10153461  -4.46209117  -3.88447271  -3.34512716  -2.83180379
  -2.33701621  -1.85567704  -1.38403959   -.91915147   -.45853835    .00000000
    .45853835    .91915147   1.38403959   1.85567704   2.33701621   2.83180379
   3.34512716   3.88447271   4.46209117   5.10153461   5.86430950    .00000000
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -6.01592556  -5.25938293  -4.62566276  -4.05366440  -3.52000681  -3.01254614
  -2.52388102  -2.04900357  -1.58425001  -1.12676082   -.67417111   -.22441455
    .22441455    .67417111   1.12676082   1.58425001   2.04900357   2.52388102
   3.01254614   3.52000681   4.05366440   4.62566276   5.25938293   6.01592556
    .00000000    .00000000    .00000000    .00000000    .00000000    .00000000
  -6.16427243  -5.41363636  -4.78532037  -4.21860944  -3.69028288  -3.18829492
  -2.70532024  -2.23642013  -1.77800112  -1.32728070   -.88198276   -.44014730
    .00000000    .44014730    .88198276   1.32728070   1.77800112   2.23642013
   2.70532024   3.18829492   3.69028288   4.21860944   4.78532037   5.41363636
   6.16427243    .00000000    .00000000    .00000000    .00000000    .00000000
  -6.30955039  -5.56452498  -4.94132496  -4.37960266  -3.85628842  -3.35942718
  -2.88176222  -2.41841576  -1.96585479  -1.52136152  -1.08273301   -.64809521
   -.21577786    .21577786    .64809521   1.08273301   1.52136152   1.96585479
   2.41841576   2.88176222   3.35942718   3.85628842   4.37960266   4.94132496
   5.56452498   6.30955039    .00000000    .00000000    .00000000    .00000000
  -6.45194014  -5.71225555  -5.09391000  -4.53690666  -4.01831867  -3.52627534
  -3.05358242  -2.59541634  -2.14829665  -1.70956074  -1.27706682   -.84901134
   -.42380790    .00000000    .42380790    .84901134   1.27706682   1.70956074
   2.14829665   2.59541634   3.05358242   3.52627534   4.01831867   4.53690666
   5.09391000   5.71225555   6.45194014    .00000000    .00000000    .00000000
  -6.59160544  -5.85701464  -5.24328537  -4.69075652  -4.17663674  -3.68913424
  -3.22111208  -2.76779535  -2.32574984  -1.89236050  -1.46553726  -1.04353527
   -.62483672   -.20806738    .20806738    .62483672   1.04353527   1.46553726
   1.89236050   2.32574984   2.76779535   3.22111208   3.68913424   4.17663674
   4.69075652   5.24328537   5.85701464   6.59160544    .00000000    .00000000
  -6.72869520  -5.99897129  -5.38964052  -4.84136365  -4.33147829  -3.84826679
  -3.38464514  -2.93588250  -2.49858569  -2.07018108  -1.64862291  -1.23221576
   -.81949868   -.40916464    .00000000    .40916464    .81949868   1.23221576
   1.64862291   2.07018108   2.49858569   2.93588250   3.38464514   3.84826679
   4.33147829   4.84136365   5.38964052   5.99897129   6.72869520    .00000000
  -6.86334529  -6.13827922  -5.53314715  -4.98891897  -4.48305536  -4.00390860
  -3.54444387  -3.09997053  -2.66713212  -2.24339147  -1.82674114  -1.41552780
  -1.00833827   -.60392106   -.20112858    .20112858    .60392106   1.00833827
   1.41552780   1.82674114   2.24339147   2.66713212   3.09997053   3.54444387
   4.00390860   4.48305536   4.98891897   5.53314715   6.13827922   6.86334529